In [1]:
import os
import xarray as xr
import oggm
from oggm import utils,cfg, workflow,tasks
from oggm.workflow import execute_entity_task
import pandas as pd
import geopandas as gpd
import numpy as np
from time import gmtime, strftime
from cluster import read_cmip6_data

In [3]:
def read_cmip6_data(path, gdirs, reset=False):
    l = ['CRU']
    for file in os.listdir(os.path.join(path,'tas')):
        if file.endswith('.nc'):
            name = file.split('_')[0]
            suffix = name.split('.')[2]
            tas_file = os.path.join(path, 'tas', file)
            pr_file = os.path.join(path, 'pr', name + '_pr.nc')
            if reset:
                try:
                    execute_entity_task(tasks.process_cmip_data, gdirs, filesuffix=suffix, fpath_temp=tas_file,
                                        fpath_precip=pr_file)
                    l.append(suffix)
                except:
                    print(suffix,'failed')
            else:
                l.append(suffix)
    return l


In [2]:
REGION=11
JOB_NR=2000
GEOD_CALIB=True
cmip6_path='cmip6_gn'

cfg.initialize()
cfg.PATHS['working_dir'] = os.path.join('test_cmip6_gn')
utils.mkdir(cfg.PATHS['working_dir'])

# Use multiprocessing?
cfg.PARAMS['use_multiprocessing'] = True

# store model geometry
cfg.PARAMS['store_model_geometry'] = True

# RGI file
path = utils.get_rgi_region_file(REGION, version='61')
rgidf = gpd.read_file(path)
rgidf = rgidf.sort_values('Area', ascending=True)

# exclude non-landterminating glaciers
rgidf = rgidf[rgidf.TermType == 0]
rgidf = rgidf[rgidf.Connect != 2]

if GEOD_CALIB:
    cfg.PARAMS['border'] = 240
    cfg.PARAMS['climate_qc_months']=0
    cfg.PARAMS['hydro_month_nh']=1
    cfg.PARAMS['hydro_month_sh']=1

    prepro_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/CRU/elev_bands/qc0/pcp2.5/match_geod_pergla/'
    url = os.path.join(prepro_url, 'RGI62/b_240/L5/summary/')
else:
    cfg.PARAMS['border'] = 160
    prepro_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/elev_bands/qc3/pcp2.5/no_match/'
    url = os.path.join(prepro_url, 'RGI62/b_160/L5/summary/')

# exculde glaciers that failed during preprocessing
fpath = utils.file_downloader(url + f'glacier_statistics_{REGION}.csv')
stat = pd.read_csv(fpath, index_col=0, low_memory=False)
rgidf = rgidf[~rgidf.RGIId.isin(stat.error_task.dropna().index)].reset_index()

#select glacier by JOB_NR
rgidf = rgidf.iloc[[JOB_NR]]

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(rgidf, from_prepro_level=5, prepro_base_url=prepro_url)
gdir=gdirs[0]
# read (reset=False) or process cmip6 data (reset=True)
gcm_list_gr = read_cmip6_data('cmip6_merge', gdirs, reset=True)
gdir

2022-01-26 12:12:37: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2022-01-26 12:12:37: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2022-01-26 12:12:37: oggm.cfg: Multiprocessing: using all available processors (N=32)
2022-01-26 12:12:37: oggm.cfg: Multiprocessing switched ON after user settings.
2022-01-26 12:12:37: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2022-01-26 12:12:37: oggm.cfg: PARAMS['border'] changed from `40` to `240`.
2022-01-26 12:12:37: oggm.cfg: PARAMS['climate_qc_months'] changed from `3` to `0`.
2022-01-26 12:12:37: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2022-01-26 12:12:37: oggm.cfg: PARAMS['hydro_month_sh'] changed from `4` to `1`.
2022-01-26 12:12:37: oggm.utils: No known hash for cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/CRU/elev_bands/qc0/pcp2.5/match_geod_pergla/RGI62/b_240/L5/summary/glacier_statistics_11.csv
2022-01-26 12

<oggm.GlacierDirectory>
  RGI id: RGI60-11.02084
  Region: 11: Central Europe
  Subregion: 11-01: Alps                            
  Glacier type: Glacier
  Terminus type: Land-terminating
  Status: Glacier or ice cap
  Area: 0.08 km2
  Lon, Lat: (9.90733, 46.3985)
  Grid (nx, ny): (508, 505)
  Grid (dx, dy): (14.0, -14.0)

In [4]:
gcm_list_gn = read_cmip6_data(cmip6_path,gdirs,reset=True)

2022-01-26 12:17:54: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:17:54: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataMPI-ESM1-2-HR
2022-01-26 12:18:02: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataMPI-ESM1-2-HR
2022-01-26 12:18:02: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:18:02: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataACCESS-CM2
2022-01-26 12:18:05: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataACCESS-CM2
2022-01-26 12:18:08: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:18:08: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataACCESS-ESM1-5
2022-01-26 12:18:08: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataACCESS-ESM1-5
2022-01-26 12:18:11: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:18:11: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataFIO-ESM-2-0
202

NorESM2-LM failed


2022-01-26 12:18:53: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataMPI-ESM-1-2-HAM
2022-01-26 12:18:55: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:18:55: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataNorESM2-MM
2022-01-26 12:18:55: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataNorESM2-MM
2022-01-26 12:18:55: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:18:55: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataCAS-ESM2-0
2022-01-26 12:18:58: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataCAS-ESM2-0
2022-01-26 12:19:02: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:02: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataFGOALS-g3
2022-01-26 12:19:04: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataFGOALS-g3
2022-01-26 12:19:06: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:

ICON-ESM-LR failed


2022-01-26 12:19:17: oggm.shop.gcm_climate: ValueError occurred during task process_cmip_dataAWI-CM-1-1-MR on RGI60-11.02084: operands could not be broadcast together with shapes (1980,) (1968,) 
2022-01-26 12:19:17: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:17: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataMPI-ESM1-2-LR


AWI-CM-1-1-MR failed


2022-01-26 12:19:18: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataMPI-ESM1-2-LR
2022-01-26 12:19:18: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:18: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataMIROC6
2022-01-26 12:19:18: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataMIROC6
2022-01-26 12:19:19: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:19: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataCMCC-ESM2
2022-01-26 12:19:19: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataCMCC-ESM2
2022-01-26 12:19:19: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:19: oggm.shop.gcm_climate: (RGI60-11.02084) process_cmip_dataCESM2
2022-01-26 12:19:20: oggm.shop.gcm_climate: (RGI60-11.02084) process_gcm_dataCESM2
2022-01-26 12:19:20: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2022-01-26 12:19:20: oggm.shop.gcm

In [8]:
for i, gcm in enumerate(gcm_list[1:]):
    print(gcm)
    if gcm != 'CRU':
        climate_filename='gcm_data'
        input_suffix=gcm
    else:
        climate_filename='climate_historical'
        input_suffix=None
    seed=1
    try:
        c = xr.open_dataset(gdir.get_filepath(climate_filename, filesuffix=input_suffix))
        years =  range(c.time.to_series().iloc[0].year + 16, c.time.to_series().iloc[-1].year - 14)
        yr=years[-1]
        mod = tasks.run_random_climate(gdir, climate_filename=climate_filename, climate_input_filesuffix=input_suffix, y0=yr,
                                                 nyears=200, unique_samples=True, output_filesuffix=gcm + '_' + str(yr),
                                                 seed=seed)
    except:
        print(gcm,'failed')

2022-01-25 15:59:15: oggm.core.flowline: (RGI60-11.02084) run_random_climateMPI-ESM1-2-HR_2019
2022-01-25 15:59:15: oggm.core.flowline: (RGI60-11.02084) flowline_model_runMPI-ESM1-2-HR_2019


MPI-ESM1-2-HR


2022-01-25 15:59:19: oggm.core.flowline: (RGI60-11.02084) run_random_climateACCESS-CM2_2019
2022-01-25 15:59:19: oggm.core.flowline: (RGI60-11.02084) flowline_model_runACCESS-CM2_2019


ACCESS-CM2


2022-01-25 15:59:23: oggm.core.flowline: (RGI60-11.02084) run_random_climateACCESS-ESM1-5_1999
2022-01-25 15:59:23: oggm.core.flowline: (RGI60-11.02084) flowline_model_runACCESS-ESM1-5_1999


...

CanESM5


2022-01-25 16:01:27: oggm.core.flowline: (RGI60-11.02084) run_random_climateBCC-ESM1_1999
2022-01-25 16:01:27: oggm.core.flowline: (RGI60-11.02084) flowline_model_runBCC-ESM1_1999


BCC-ESM1


2022-01-25 16:01:32: oggm.core.flowline: (RGI60-11.02084) run_random_climateGISS-E2-1-G_1999
2022-01-25 16:01:32: oggm.core.flowline: (RGI60-11.02084) flowline_model_runGISS-E2-1-G_1999


GISS-E2-1-G


In [7]:
def analyze_cmip6_data(path, gdirs):
    df = pd.DataFrame()
    for file in os.listdir(os.path.join(path,'tas')):
        if file.endswith('.nc'):
            name = file.split('_')[0]
            suffix = name.split('.')[2]
            tas_file = os.path.join(path, 'tas', file)
            pr_file = os.path.join(path, 'pr', name + '_pr.nc')
            ds = xr.open_dataset(tas_file)
            try:

                df = df.append({'institution_id':name.split('.')[1],
                                'source_id':name.split('.')[2],
                                'grid':name.split('.')[-1],
                                'realization':'r1i1p1f1',
                                r'No. of grids (Lat x Lon)':str(int(ds.dims['lat']))+' x '+ str(int(ds.dims['lon'])),
                                }, ignore_index=True)
            except:
                df = df.append({'institution_id':name.split('.')[1],
                                'source_id':name.split('.')[2],
                                'grid':'gn',
                                'realization':'r1i1p1f1'}, ignore_index=True)
    return df 

In [8]:
df_gn =analyze_cmip6_data(cmip6_path, gdirs)

In [9]:
df_gr = analyze_cmip6_data('cmip6_merge', gdirs)

In [19]:
df = df_gn.append(df_gr,ignore_index=True).sort_values(by='institution_id').reset_index(drop=True)

In [20]:
df

,institution_id,source_id,grid,realization,No. of grids (Lat x Lon)
0,AS-RCEC,TaiESM1,gn,r1i1p1f1,192 x 288
1,AWI,AWI-CM-1-1-MR,gn,r1i1p1f1,192 x 384
2,AWI,AWI-ESM-1-1-LR,gn,r1i1p1f1,96 x 192
3,BCC,BCC-ESM1,gn,r1i1p1f1,64 x 128
4,BCC,BCC-CSM2-MR,gn,r1i1p1f1,160 x 320
5,CAMS,CAMS-CSM1-0,gn,r1i1p1f1,160 x 320
6,CAS,FGOALS-g3,gn,r1i1p1f1,80 x 180
7,CAS,FGOALS-f3-L,gr,r1i1p1f1,180 x 288
8,CAS,CAS-ESM2-0,gn,r1i1p1f1,128 x 256
9,CCCR-IITM,IITM-ESM,gn,r1i1p1f1,94 x 192


In [21]:
for i, suffix in df.source_id.iteritems():
    p = gdir.get_filepath('gcm_data',filesuffix=suffix)
    if os.path.isfile(p):
        t_ds = xr.open_dataset(p)
        df.loc[i,'time']=str(t_ds.attrs['hydro_yr_0'])+'-'+str(t_ds.attrs['hydro_yr_1'])
    else:
        print(suffix)
df = df.sort_values(by='institution_id').reset_index(drop=True)

AWI-CM-1-1-MR
ICON-ESM-LR
NorESM2-LM


In [22]:
df.time.value_counts()

1850-2034    22
1850-2014    21
1850-2016     1
1850-1999     1
Name: time, dtype: int64

In [26]:
df.to_csv('cmip6_historical_overview.csv',sep=',')